In [ ]:
# Define model
def DefineModel1(Model):
    Model.Racks = pyo.Var(domain = pyo.PositiveIntegers, initialize = int(len(Model.P)/(Model.MaxShelves / 2 / Model.PalletsPerShelf)))   # Number of racks
    Model.Shelf = pyo.Var(Model.S, within = pyo.Binary, initialize = 0)   # Shelf is included in a rack
    Model.ShelfHeights = pyo.Var(Model.S, within = pyo.NonNegativeIntegers, bounds=(0, Model.MaxShelfSize), initialize = Model.MaxShelfSize)   # Height of each shelf
    Model.Allocation = pyo.Var(Model.P, Model.S, domain = pyo.Binary, initialize = 0)   # Allocation of pallets to shelves

    def rule_fit(Model, P):   # Each pallet must be allocated to a shelf that is at least the height of the pallet
        return sum(Model.ShelfHeights[s] * Model.Allocation[P, s] for s in Model.S) >= Model.Pallets[P]
    Model.PalletFits = pyo.Constraint(Model.P, rule = rule_fit)
    
    def rule_use(Model, P):   # Each pallet must be allocated to exactly one shelf
        return sum(Model.Allocation[P, s] for s in Model.S) == 1
    Model.MustUse = pyo.Constraint(Model.P, rule = rule_use)

    def rule_within(Model, S):   # For each shelf, the allocation must be within the number of available slots.
        return sum(Model.Allocation[p, S] for p in Model.P) <= Model.PalletsPerShelf * Model.Racks   # Some shelves may be empty (Allocation = 0)
    Model.WithinRack = pyo.Constraint(Model.S, rule = rule_within)

    def rule_height(Model):   # Total height of shelves in a rack, plus gaps, must equal the rack height
        return sum(Model.ShelfHeights[s] + Model.Shelf[s] * Model.Gap for s in Model.S) == Model.RackHeight
    Model.RackTotalHeight = pyo.Constraint(rule = rule_height)

    def rule_size(Model, S):   # Each shelf can be no higher than the maximum. Some heights may be zero
        return Model.ShelfHeights[S] <= Model.Shelf[S] * Model.MaxShelfSize
    Model.ShelfSize = pyo.Constraint(Model.S, rule = rule_size)

    def rule_shelflink(Model, S):   # Indicate that we're using an available shelf slot only if we allocate a pallet to that slot
        return sum(Model.Allocation[p, S] for p in Model.P) >= Model.Shelf[S]
    Model.ShelfUB = pyo.Constraint(Model.S, rule = rule_shelflink)

    def rule_shelfSymmetry(Model, S):   # Break symmetry by ordering shelf sizes
        if S == Model.S.last():  # Skip for last shelf, since we're looking forward to the next shelf
            return pyo.Constraint.Skip
        else:
            return Model.ShelfHeights[S] >= Model.ShelfHeights[Model.S.next(S)]   # [Model.S.next(S)] is equivalent to [S + 1]
    Model.shelfSymmetry = pyo.Constraint(Model.S, rule = rule_shelfSymmetry)
    
    def rule_Obj(Model):   # Minimize the number of racks we need to allocate all pallets to a shelf
        if WeightedObj:   # Weighted to also minimize number of shelves in a rack, if required
            return Model.WeightRacks * Model.Racks + Model.WeightShelves * sum(Model.Shelf[s] for s in Model.S)
        else:
            return Model.Racks
    Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.minimize)